In [1]:
using Plots
using DSP
using Random

function generate_pss_sequence()
    # Generate PSS sequence (assuming BPSK modulation)
    pss_symbols = rand([-1.0, 1.0], 127)
    
    # Assuming subcarrier placement for PSS
    subcarrier_start = 56
    subcarrier_end = subcarrier_start + 127 - 1
    
    pss_sequence = zeros(Complex{Float64}, 336)
    
    for i in 1:127
        pss_sequence[subcarrier_start + i - 1] = pss_symbols[i]
    end
    
    return pss_sequence
end

function generate_sss_sequence(NID2)
    # Generate SSS sequence (assuming BPSK modulation)
    sss_symbols = rand([-1.0, 1.0], 127)
    
    # Assuming subcarrier placement for SSS
    subcarrier_start = 56
    subcarrier_end = subcarrier_start + 127 - 1
    
    sss_sequence = zeros(Complex{Float64}, 336)
    
    for i in 1:127
        sss_sequence[subcarrier_start + i - 1] = sss_symbols[i]
    end
    
    return sss_sequence
end

function simulate_channel(input_signal, snr_dB, frequency_offset)
    # Simulate a multipath channel with noise and frequency offset
    
    # Add frequency offset
    phase_offset = exp.(1.0im * 2π * frequency_offset * (1:length(input_signal)))
    signal_with_offset = input_signal .* phase_offset
    
    # Simulate multipath channel (Rayleigh fading)
    channel_response = randn(Complex{Float64}, length(input_signal)) + 1.0im * randn(Complex{Float64}, length(input_signal))
    
    # Add noise based on SNR
    noise_power = 10^(-snr_dB / 20)
    noise = sqrt(noise_power / 2) * (randn(Complex{Float64}, length(input_signal)) + 1.0im * randn(Complex{Float64}, length(input_signal)))
    
    output_signal = conv(signal_with_offset, channel_response) + noise
    
    return output_signal
end


# Example usage
NID2 = 123  # Replace with the actual NID2 value
pss_sequence = generate_pss_sequence()
sss_sequence = generate_sss_sequence(NID2)

# Combine PSS and SSS sequences
combined_sequence = pss_sequence + sss_sequence

# Simulate channel with noise and frequency offset
snr_dB = 20
frequency_offset = 1.5e6  # Frequency offset in Hz
received_signal = simulate_channel(combined_sequence, snr_dB, frequency_offset)

# Plot the original and received signals
plot(abs.(combined_sequence), label="Original Signal")
plot!(abs.(received_signal), label="Received Signal")
title!("PSS + SSS with Noise, Multipath Channel, and Frequency Offset")
xlabel!("Subcarrier Index")
ylabel!("Amplitude")


LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(671),), b has dims (Base.OneTo(336),), mismatch at 1